# Simple jupyter notebook for accesing TimescaleDB database via kubectl port forward for SQL query

*Aniket Pant, UIUC*

In [2]:
import os

import pandas as pd
from sqlalchemy import create_engine, text
from sqlalchemy.engine import URL

In [3]:
# Build a SQLAlchemy engine using Postgres connection details from the environment.
db_url = URL.create(
    "postgresql+psycopg2",
    username=os.getenv("POSTGRES_USER", "ships"),
    password=os.getenv("POSTGRES_PASSWORD", "ships"),
    host=os.getenv("POSTGRES_HOST", "localhost"),
    port=int(os.getenv("POSTGRES_PORT", 5432)),
    database=os.getenv("POSTGRES_DB", "ships"),
)

engine = create_engine(db_url, future=True)

with engine.connect() as connection:
    df = pd.read_sql(text("SELECT NOW() AS server_time"), connection)

In [4]:
df

,server_time
0,2025-10-08 05:44:50.369433+00:00
